# Hyperlocal Logistics Benchmarking & Delay Classification System

This notebook demonstrates a supervised machine learning model
that predicts delivery delay risk using hyperlocal, company-specific
historical logistics data.

Target Users:
- Small & Medium Businesses (SMEs)
- Large Enterprises

Model Type: Classification  
Algorithm: Random Forest

In [12]:
import pandas as pd
import numpy as np

# Create dummy hyperlocal logistics dataset
np.random.seed(42)

df = pd.DataFrame({
    "distance_km": np.random.randint(1, 50, 300),
    "traffic_level": np.random.choice([0, 1, 2], 300),  # 0=Low,1=Medium,2=High
    "weather_condition": np.random.choice([0, 1, 2], 300),  # 0=Clear,1=Rain,2=Storm
    "route_on_time_rate": np.random.uniform(0.5, 1.0, 300),
    "delivery_risk": np.random.choice([0, 1], 300)  # target
})

print("Dataset ready:", df.shape)

Dataset ready: (300, 5)


In [ ]:
# Basic check to ensure Colab environment is ready
print("Colab environment is ready. ML model build starts now.")

Colab environment is ready. ML model build starts now.


In [ ]:
# Core data & ML libraries
import pandas as pd
import numpy as np

# ML utilities
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

# Evaluation
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# Fix randomness for reproducibility (pro habit)
np.random.seed(42)

# Number of delivery records (SME + scalable)
n_samples = 300

# Simulated hyperlocal logistics data
data = {
    "distance_km": np.random.randint(2, 50, n_samples),
    "route_id": np.random.choice(["R1", "R2", "R3", "R4", "R5"], n_samples),
    "road_type": np.random.choice(["City", "Highway", "Mixed"], n_samples),
    "time_of_day": np.random.choice(["Morning", "Afternoon", "Evening", "Night"], n_samples),
    "day_type": np.random.choice(["Weekday", "Weekend"], n_samples),
    "traffic_level": np.random.choice(["Low", "Medium", "High"], n_samples),
    "weather_condition": np.random.choice(["Clear", "Rain", "Storm"], n_samples),
}

df = pd.DataFrame(data)
df.head()

,distance_km,route_id,road_type,time_of_day,day_type,traffic_level,weather_condition
0,40,R2,Highway,Afternoon,Weekend,Low,Clear
1,30,R2,City,Evening,Weekend,High,Storm
2,16,R3,Mixed,Evening,Weekend,Medium,Clear
3,44,R5,Mixed,Evening,Weekend,Medium,Clear
4,9,R1,City,Night,Weekday,Low,Storm


In [ ]:
# Route-level historical benchmarks (company-specific learning)
route_avg_delay = {
    "R1": 5,
    "R2": 12,
    "R3": 8,
    "R4": 18,
    "R5": 10
}

route_on_time_rate = {
    "R1": 0.92,
    "R2": 0.75,
    "R3": 0.85,
    "R4": 0.60,
    "R5": 0.80
}

df["route_avg_delay_minutes"] = df["route_id"].map(route_avg_delay)
df["route_on_time_rate"] = df["route_id"].map(route_on_time_rate)

df.head()

,distance_km,route_id,road_type,time_of_day,day_type,traffic_level,weather_condition,route_avg_delay_minutes,route_on_time_rate
0,40,R2,Highway,Afternoon,Weekend,Low,Clear,12,0.75
1,30,R2,City,Evening,Weekend,High,Storm,12,0.75
2,16,R3,Mixed,Evening,Weekend,Medium,Clear,8,0.85
3,44,R5,Mixed,Evening,Weekend,Medium,Clear,10,0.80
4,9,R1,City,Night,Weekday,Low,Storm,5,0.92


In [ ]:
# Create delivery risk label
# 0 = On-Time, 1 = Delayed

df["delivery_risk"] = (
    (df["traffic_level"] == "High") |
    (df["weather_condition"] == "Storm") |
    (df["route_on_time_rate"] < 0.7)
).astype(int)

df[["traffic_level", "weather_condition", "route_on_time_rate", "delivery_risk"]].head()

,traffic_level,weather_condition,route_on_time_rate,delivery_risk
0,Low,Clear,0.75,0
1,High,Storm,0.75,1
2,Medium,Clear,0.85,0
3,Medium,Clear,0.80,0
4,Low,Storm,0.92,1


In [ ]:
# Encode categorical features
encoder = LabelEncoder()

categorical_cols = [
    "route_id",
    "road_type",
    "time_of_day",
    "day_type",
    "traffic_level",
    "weather_condition"
]

for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])

df.head()


,distance_km,route_id,road_type,time_of_day,day_type,traffic_level,weather_condition,route_avg_delay_minutes,route_on_time_rate,delivery_risk
0,40,1,1,0,1,1,0,12,0.75,0
1,30,1,0,1,1,0,2,12,0.75,1
2,16,2,2,1,1,2,0,8,0.85,0
3,44,4,2,1,1,2,0,10,0.80,0
4,9,0,0,3,0,1,2,5,0.92,1


In [ ]:
# Split features and target
X = df.drop("delivery_risk", axis=1)
y = df["delivery_risk"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])

Training samples: 225
Testing samples: 75


In [13]:
from sklearn.model_selection import train_test_split

# X = features, y = target
X = df.drop("delivery_risk", axis=1)
y = df["delivery_risk"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42
)

In [14]:
from sklearn.ensemble import RandomForestClassifier
# Train Random Forest Classifier
model = RandomForestClassifier(
    n_estimators=150,
    max_depth=8,
    random_state=42
)

model.fit(X_train, y_train)

print("Model training completed.")

Model training completed.


In [ ]:
# Predict on test data
y_pred = model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)

Model Accuracy: 1.0


In [ ]:
print("Classification Report:\n")
print(classification_report(y_test, y_pred))

Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        34
           1       1.00      1.00      1.00        41

    accuracy                           1.00        75
   macro avg       1.00      1.00      1.00        75
weighted avg       1.00      1.00      1.00        75



In [ ]:
confusion_matrix(y_test, y_pred)

array([[34,  0],
       [ 0, 41]])

In [ ]:
feature_importance = pd.DataFrame({
    "feature": X.columns,
    "importance": model.feature_importances_
}).sort_values(by="importance", ascending=False)

feature_importance

,feature,importance
6,weather_condition,0.359437
5,traffic_level,0.359203
7,route_avg_delay_minutes,0.095803
8,route_on_time_rate,0.077147
0,distance_km,0.045332
1,route_id,0.026234
3,time_of_day,0.017263
2,road_type,0.012164
4,day_type,0.007418


In [ ]:
import joblib
joblib.dump(model, "hyperlocal_logistics_model.pkl")

print("Model saved successfully.")

Model saved successfully.


In [ ]:
# New delivery scenario (demo input)

new_delivery = pd.DataFrame({
    "distance_km": [28],
    "route_id": [2],            # encoded value
    "road_type": [1],           # encoded value
    "time_of_day": [3],         # encoded value
    "day_type": [0],            # encoded value
    "traffic_level": [2],       # encoded value (High)
    "weather_condition": [1],   # encoded value (Rain)
    "route_avg_delay_minutes": [12],
    "route_on_time_rate": [0.75]
})

new_delivery

,distance_km,route_id,road_type,time_of_day,day_type,traffic_level,weather_condition,route_avg_delay_minutes,route_on_time_rate
0,28,2,1,3,0,2,1,12,0.75


In [ ]:
prediction = model.predict(new_delivery)

if prediction[0] == 0:
    print("✅ Prediction: On-Time Delivery")
else:
    print("⚠️ Prediction: High Risk of Delay")

✅ Prediction: On-Time Delivery


## Live Prediction Demo

This section demonstrates how the trained ML model predicts
delivery risk for a new delivery request in real time.

In [ ]:
import joblib

# Save the trained model
joblib.dump(model, "hyperlocal_logistics_model.pkl")

print("✅ Model saved successfully as hyperlocal_logistics_model.pkl")

✅ Model saved successfully as hyperlocal_logistics_model.pkl


In [ ]:
import joblib

# Save the trained model
joblib.dump(model, "hyperlocal_logistics_model.pkl")

print("✅ Model saved successfully as hyperlocal_logistics_model.pkl")

✅ Model saved successfully as hyperlocal_logistics_model.pkl


In [ ]:
import joblib

joblib.dump(model, "hyperlocal_logistics_model.pkl")

print("✅ Model saved successfully")

✅ Model saved successfully


In [1]:
import joblib
joblib.dump(model, "hyperlocal_logistics_model.pkl")

NameError: name 'model' is not defined

In [2]:
print(type(ML_Model))

NameError: name 'ML_Model' is not defined

In [3]:
print(locals().keys())


dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'joblib', '_i2', '_i3'])


In [15]:
import joblib
joblib.dump(model, "hyperlocal_logistics_model.pkl")

['hyperlocal_logistics_model.pkl']